In [1]:
import json
from datetime import datetime
from pprint import pprint

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from tqdm.notebook import tqdm

In [2]:
from heart import *
from hotel import *
from airline import *

data_and_transformers = [
    ('heart', load_heart(), transformers_heart()),
    ('airline', load_airline(), transformers_airline()),
    ('hotel', load_hotel(), transformers_hotel())
]

In [3]:
from models import models_and_param_grids

In [4]:
results = []

for data_name, data, transformers in tqdm(data_and_transformers):
    for model_name, (model, param_grid) in tqdm(models_and_param_grids.items()):

        pipeline = make_pipeline(*transformers, model)
        grid_search = GridSearchCV(
            estimator=pipeline,
            cv=StratifiedKFold(n_splits=5),
            param_grid=param_grid,
            n_jobs=3,
            verbose=1,
            scoring=['accuracy', 'f1', 'precision', 'recall'],
            refit='f1',
            error_score='raise'
        )
        display(grid_search)

        X_train, X_test, y_train, y_test = data
        grid_search.fit(X=X_train, y=y_train)
        y_pred = grid_search.predict(X_test)

        res = {
            'dataset': data_name,
            'model': model_name,
            'f1': f1_score(y_test, y_pred),
            'accuracy': accuracy_score(y_test, y_pred)
        }
        display(pd.DataFrame([res]))
        with open(f'./output/{datetime.now()}_{model_name}_{data_name}.json', 'wt') as f:
            json.dump(res, f)
        results.append(res)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['Diabetes']),
                                                                        ('kbinsdiscretizer',
                                                                         KBinsDiscretizer(encode='onehot-dense',
                                                                                          random_state=0,
                                                                                          strategy='uniform',
                                                                                          subsample=None),
                                                                         ['Age',
                                                                          'Education',
                                                                          'Income',
                                                                          'BMI',
                                                                          'MentHlth',
                                                                          'PhysHlth',
                                                                          'GenHlth'])])),
                                       ('bernoullinb',
                                        BernoulliNB(binarize=False))]),
             n_jobs=3, param_grid={'bernoullinb__alpha': [1.0]}, refit='f1',
             scoring=['accuracy', 'f1', 'precision', 'recall'], verbose=1)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


,dataset,model,f1,accuracy
0,heart,nb,0.384615,0.84


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


,dataset,model,f1,accuracy
0,hotel,knn,0.555556,0.72


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise',
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('simpleimputer',
                                                                         SimpleImputer(fill_value=0,
                                                                                       strategy='constant'),
                                                                         ['children'])],
                                                          verbose_feature_names_out=False)),
                                       ('columntransformer-2',
                                        ColumnTransf...
                                                                          'stays_in_weekend_nights',
                                                                          'hotel',
                                                                          'country',
                                                                          'distribution_channel',
                                                                          'is_repeated_guest',
                                                                          'previous_bookings_not_canceled',
                                                                          'previous_cancellations'])])),
                                       ('catboostclassifier',
                                        <catboost.core.CatBoostClassifier object at 0x7fd8d7e5cbb0>)]),
             n_jobs=3, param_grid={'catboostclassifier__n_estimators': [100]},
             refit='f1', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=1)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


,dataset,model,f1,accuracy
0,hotel,catboost,0.619048,0.76


In [5]:
df_results = pd.DataFrame(results).pivot(
    index='model',
    columns='dataset',
    values=['f1', 'accuracy'],
).swaplevel(axis=1).sort_index(axis=1)
df_results.style.highlight_max().format(precision=3)

In [6]:
df_results.style.highlight_max(props='textbf:--rwrap;').format(precision=3).to_latex('table.tex')